<a href="https://colab.research.google.com/github/shashankdeshpande/llm-examples/blob/master/OpenAI_API_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI - Function calling capability

OpenAI has recently introduced a new capability called function calling, which allows developers to describe functions to GPT-4 and GPT-3.5 and have the models create code to execute those functions

This feature enables chatbots to leverage external tools, transform natural language into database queries, and extract structured data from text. The models have been fine-tuned to identify instances where a function should be invoked and provide JSON responses that align with the function signature. Function calling allows developers to more reliably get structured data back from the model

**Reference** - https://openai.com/blog/function-calling-and-other-api-updates

In [1]:
!pip install -q openai dateparser

In [2]:
import json
import openai
import logging
import requests
import traceback
import dateparser

In [12]:
logger = logging.getLogger('OpenAI-Example')
logger.setLevel(logging.INFO)

In [34]:
# https://platform.openai.com/account/api-keys

openai.api_key = "YOUR_OPENAI_KEY_HERE"

## Step 1 - Create a custom function
In this step, we will be creating a personalized function that will retrieve country-specific trends upon request.

We are referring google trends from here - https://trends.google.com/trends/trendingsearches/daily?geo=IN&hl=en-GB

In [13]:
def get_google_trends(country_code, date_string='today'):
    """Method to retrieve country specific trends

    Args:
        country_code (str): alpha-2 country code, example - IN,US,UK
        date_string (str): date string supported by dateparser, example - today, yesterday

    Returns:
        list: list of dict having trend query and traffic details
    """
    trends = []
    try:
        logger.info(f'received args - country_code: {country_code}, date_string: {date_string}')
        dt = dateparser.parse(date_string).strftime('%Y%m%d')
        url = 'https://trends.google.com/trends/api/dailytrends'
        params = {'hl':'en-GB','tz':-330, 'geo': country_code, 'ns':15, 'ed': dt}
        response = requests.get(url, params=params, timeout=5)
        if response.status_code == 200:
            tinfo = json.loads(response.content[5:])
            tinfo = tinfo['default']['trendingSearchesDays'][0]
            if tinfo['date'] == dt:
                for i in tinfo['trendingSearches']:
                    trends.append({
                        'query': i['title']['query'],
                        'traffic': i['formattedTraffic']
                    })
    except Exception as e:
        logger.error('Error in get_trends: \n {}'.format(traceback.format_exc()))
    return trends

In [14]:
get_google_trends('IN')

INFO:OpenAI-Example:received args - country_code: IN, date_string: today


[{'query': 'Messi', 'traffic': '100K+'},
 {'query': 'Wagner Group', 'traffic': '20K+'},
 {'query': 'Lionel Messi', 'traffic': '20K+'},
 {'query': 'Rahul Gandhi', 'traffic': '20K+'},
 {'query': 'Ruturaj Gaikwad', 'traffic': '10K+'},
 {'query': 'TSPSC Group 4 Hall Ticket', 'traffic': '10K+'},
 {'query': 'Mary Millben', 'traffic': '10K+'},
 {'query': 'Lionel Messi Birthday', 'traffic': '5K+'}]

## Step 2 - Specify necessary function details as required by OpenAI
Make sure to mention clear and detailed description for function as well as arguments required by that function. This will assist the OpenAI model in determining the appropriate usage of the function and aid in extracting the essential entities from user queries.

We can have details of multiple functions, currently experimenting with only one.

In [16]:
custom_function_info = [
    {
        "name": "get_google_trends",
        "description": "Get list of trending queries along with traffic observed",
        "parameters": {
            "type": "object",
            "properties": {
                "country_code": {
                    "type": "string",
                    "description": "alpha-2 country code",
                },
                "date_string": {
                    "type": "string",
                    "description": "date string supported by dateparser"
                    },
            },
            "required": ["country_code", "date_string"],
        },
    }
]

## Step 3 - Call OpenAI API with custom function details

In [17]:
user_query = "Show me current google trends in India"

openai_model = "gpt-3.5-turbo-0613"

response = openai.ChatCompletion.create(
    model=openai_model,
    messages=[{"role": "user", "content": user_query}],
    functions=custom_function_info,
    function_call="auto", #OpenAI will determine whether a custom function is necessary or not based on query, otherwise can set this as {"name": "get_google_trends"}
)

In [18]:
response

<OpenAIObject chat.completion id=chatcmpl-7Utl9iv3YiXanW1Rd6dYl07TQBoKz at 0x7fb650c837e0> JSON: {
  "id": "chatcmpl-7Utl9iv3YiXanW1Rd6dYl07TQBoKz",
  "object": "chat.completion",
  "created": 1687598535,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_google_trends",
          "arguments": "{\n  \"country_code\": \"IN\",\n  \"date_string\": \"today\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 26,
    "total_tokens": 105
  }
}

## Step 4 - Retrieve identified arguments from the response and call the custom function

In [19]:
identified_func = response.choices[0].message.to_dict()
print('Response function call: ', identified_func)

identified_func_args = identified_func['function_call']['arguments']
identified_func_args = json.loads(identified_func_args)
print('Function arguments: ', identified_func_args)

Response function call:  {'role': 'assistant', 'content': None, 'function_call': <OpenAIObject at 0x7fb650d9fec0> JSON: {
  "name": "get_google_trends",
  "arguments": "{\n  \"country_code\": \"IN\",\n  \"date_string\": \"today\"\n}"
}}
Function arguments:  {'country_code': 'IN', 'date_string': 'today'}


In [20]:
func_output = get_google_trends(**identified_func_args)
func_output

INFO:OpenAI-Example:received args - country_code: IN, date_string: today


[{'query': 'Messi', 'traffic': '100K+'},
 {'query': 'Wagner Group', 'traffic': '20K+'},
 {'query': 'Lionel Messi', 'traffic': '20K+'},
 {'query': 'Rahul Gandhi', 'traffic': '20K+'},
 {'query': 'Ruturaj Gaikwad', 'traffic': '10K+'},
 {'query': 'TSPSC Group 4 Hall Ticket', 'traffic': '10K+'},
 {'query': 'Mary Millben', 'traffic': '10K+'},
 {'query': 'Lionel Messi Birthday', 'traffic': '5K+'}]

## Step 5: Call OpenAI with function output and query to get answer

In [21]:
final_response = openai.ChatCompletion.create(
    model=openai_model,
    messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": json.dumps(identified_func_args)}},
        {"role": "function", "name": "get_google_trends", "content": json.dumps(func_output)}
        ],
    functions=custom_function_info
)

In [22]:
final_response

<OpenAIObject chat.completion id=chatcmpl-7UtlQ3BqPFyZ7XNeYczOPfpSGBR6M at 0x7fb651dcb470> JSON: {
  "id": "chatcmpl-7UtlQ3BqPFyZ7XNeYczOPfpSGBR6M",
  "object": "chat.completion",
  "created": 1687598552,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Here are the current Google trends in India:\n\n1. Messi - 100K+\n2. Wagner Group - 20K+\n3. Lionel Messi - 20K+\n4. Rahul Gandhi - 20K+\n5. Ruturaj Gaikwad - 10K+\n6. TSPSC Group 4 Hall Ticket - 10K+\n7. Mary Millben - 10K+\n8. Lionel Messi Birthday - 5K+"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 252,
    "completion_tokens": 94,
    "total_tokens": 346
  }
}

In [23]:
print(final_response.choices[0]['message']['content'])

Here are the current Google trends in India:

1. Messi - 100K+
2. Wagner Group - 20K+
3. Lionel Messi - 20K+
4. Rahul Gandhi - 20K+
5. Ruturaj Gaikwad - 10K+
6. TSPSC Group 4 Hall Ticket - 10K+
7. Mary Millben - 10K+
8. Lionel Messi Birthday - 5K+


# Consolidating all these elements ..
Creating a separate function which will return respond to user query

In [24]:
def ask_openai(user_query):
    answer = ''
    response = openai.ChatCompletion.create(
        model=openai_model,
        messages=[{"role": "user", "content": user_query}],
        functions=custom_function_info,
        function_call="auto"
    )
    resp = response.choices[0].message.to_dict()
    if 'function_call' in resp:
        identified_func_args = json.loads(resp['function_call']['arguments'])
        func_output = get_google_trends(**identified_func_args)
        final_response = openai.ChatCompletion.create(
            model=openai_model,
            messages=[
                {"role": "user", "content": user_query},
                {"role": "assistant", "content": None, "function_call": {"name": resp['function_call']['name'], "arguments": json.dumps(identified_func_args)}},
                {"role": "function", "name": resp['function_call']['name'], "content": json.dumps(func_output)}
                ],
            functions=custom_function_info
        )
        answer = final_response.choices[0]['message']['content']
    else:
        answer = resp['content']
    return answer


In [33]:
answer = ask_openai('Which is the most popular search trend in the nation with the largest democracy.')
print(answer)

INFO:OpenAI-Example:received args - country_code: IN, date_string: today


The most popular search trend in India, which is the nation with the largest democracy, is "Messi" with over 100K+ searches.


In [31]:
answer = ask_openai('Could you provide information on the topic that was trending in the US yesterday and share the traffic data for the most popular trend?')
print(answer)

INFO:OpenAI-Example:received args - country_code: US, date_string: yesterday


The following topics were trending in the US yesterday:

1. Russia - 500K+ traffic
2. Kim Petras - 100K+ traffic
3. Frozen fruit recall - 50K+ traffic
4. Reds - 50K+ traffic
5. Cincinnati Reds - 50K+ traffic
6. Evil Dead Rise - 50K+ traffic
7. Jordan Walsh - 50K+ traffic
8. Damian Lillard - 20K+ traffic
9. Lonzo Ball - 20K+ traffic
10. Taylor Swift Minneapolis - 20K+ traffic
11. Highlands Ranch tornado - 20K+ traffic
12. Marcus Sasser - 20K+ traffic
13. Lab-grown meat - 20K+ traffic
14. Knicks - 20K+ traffic
15. Adam Rich - 20K+ traffic
16. Nick Smith Jr - 20K+ traffic

The most popular trend based on traffic was "Russia" with over 500K searches.


In [47]:
# Note - In this example, custom function is not utilized which is correct

answer = ask_openai('Can you explain blockchain in simple terms')
print(answer)

Certainly! Blockchain is a technology that allows multiple parties to maintain a secure and transparent record of transactions or information without the need for a central authority, such as a bank or government.

Imagine a digital ledger or spreadsheet that is shared and continuously updated across a network of computers called nodes. Each node has a copy of this ledger and verifies the transactions or information recorded on it.

When a new transaction or piece of information needs to be added to the ledger, it is grouped together with other transactions into a block. Before this block is added to the chain, it goes through a process called "consensus." This means that the nodes in the network agree that the information in the block is valid and should be added.

Once a block is approved and added to the chain, it becomes a permanent part of the ledger and cannot be altered or deleted. Each block also contains a reference to the previous block, creating a chain of linked blocks.

Th